In [1]:
import pandas as pd
import configparser
import os

# Read the value from the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')
output_file_dir = config.get('path', 'output_file_dir')
output_file_name_net_sales = config.get('path', 'output_file_name_net_sales')
cateFileDir = config.get('path', 'cateFileDir')
# Read the category file
try:
    df_cate = pd.read_excel(cateFileDir)
except FileNotFoundError as e:
    try:
        df_cate = pd.read_excel(r"/Users/trieupham/Library/CloudStorage/OneDrive-BTMGlobalConsulting/Projects/data xls/DataCate.xlsx")
    except FileNotFoundError as e:
        df_cate = pd.read_excel(r"C:\Users\Trieu.Pham\OneDrive - BTM Global Consulting\Projects\data xls\DataCate.xlsx")
# using os.path.join() method
# output_file_dir = os.path.join(output_file_dir, 'outputFile')
net_sales_data = os.path.join(output_file_dir, output_file_name_net_sales) + '.csv'
# Read the csv file
# input_file should be the path to the merge csv file
try:
    input_file = net_sales_data
    df = pd.read_csv(input_file)
except FileNotFoundError as e:
    try:
        # Using the path to the csv file for mac os
        input_file = r"/Users/trieupham/Library/CloudStorage/OneDrive-BTMGlobalConsulting/Projects/data csv/outputFile/dataNetsales.csv"
        df = pd.read_csv(input_file)
    except FileNotFoundError as e:
        # Using the path to the csv file for windows
        input_file = r"C:\Users\Trieu.Pham\OneDrive - BTM Global Consulting\Projects\data csv\outputFile\dataNetsales.csv"
        df = pd.read_csv(input_file)
df.dropna(inplace=True)

In [2]:
df_cate

,Barcode,Division Name,Category Name,SubCategory Name
0,9555715 201187,FF Onsite,Hot pot,Hot pot Thai
1,9555715 200067,FF Onsite,Hot pot,Hot pot Thai
2,8935282 907940,FoodService,Raw Material & Packages,Raw material
3,8935282 900651,FoodService,Raw Material & Packages,Raw material
4,7804350 046687,Wine&Liquor,Wine,Table/Varietal Wine
...,...,...,...,...
18598,07461381,Fresh Grocery,Vegetables Cat,Vegetables Sub
18599,07180000,Fresh Grocery,Vegetables Cat,Vegetables Sub
18600,5001019,Dessert,Dessert Cat,Dessert Sub
18601,2171586,Frozen Food,Frozen meat,Frozen meat Sub


In [3]:
df

,Date,Time,InvoiceID,Barcode,Total Include VAT
0,2023-01-01,00:40:59,VN0001010101230001,8935049500544,7000.0
1,2023-01-01,00:41:18,VN0001010101230002,8938512632025,12000.0
2,2023-01-01,00:41:55,VN0001010101230003,8936011773416,25000.0
3,2023-01-01,00:41:55,VN0001010101230003,8936079121761,13000.0
4,2023-01-01,00:41:55,VN0001010101230003,8850453017528,13000.0
...,...,...,...,...,...
68978410,2023-07-05,18:58:59,VN0061010507230308,8935250402019,48000.0
68978411,2023-07-05,13:02:59,VN0061010507230185,8934803071085,13000.0
68978412,2023-07-05,09:15:59,VN0061010507230092,8935090910873,36000.0
68978413,2023-07-05,09:15:59,VN0061010507230092,8935005800015,12000.0


In [6]:
df.drop_duplicates(inplace=True)
df

,Date,Time,InvoiceID,Barcode,Total Include VAT
0,2023-01-01,00:40:59,VN0001010101230001,8935049500544,7000.0
1,2023-01-01,00:41:18,VN0001010101230002,8938512632025,12000.0
2,2023-01-01,00:41:55,VN0001010101230003,8936011773416,25000.0
3,2023-01-01,00:41:55,VN0001010101230003,8936079121761,13000.0
4,2023-01-01,00:41:55,VN0001010101230003,8850453017528,13000.0
...,...,...,...,...,...
68978410,2023-07-05,18:58:59,VN0061010507230308,8935250402019,48000.0
68978411,2023-07-05,13:02:59,VN0061010507230185,8934803071085,13000.0
68978412,2023-07-05,09:15:59,VN0061010507230092,8935090910873,36000.0
68978413,2023-07-05,09:15:59,VN0061010507230092,8935005800015,12000.0


In [7]:
# # Merge df and df_cate
# df = df.merge(df_cate, on='Barcode', how='left')

# df

In [8]:
# Add new column StoreID (first 6 digits of InvoiceID)
df['StoreID'] = df['InvoiceID'].str[:6]

In [9]:
# # Group by InvoiceID and calculate the sum of Total Include VAT for each group
df_grouped = df.groupby('InvoiceID', as_index=False).agg({
    'StoreID': 'first',
    'Date': 'first',
    'Time': 'first',
    'Barcode': 'first',
    'Total Include VAT': 'sum'
})


In [10]:
from datetime import datetime

# Function to convert time string to a proper time format
def convert_to_time(time_str):
    try:
        return datetime.strptime(time_str, '%H:%M:%S').strftime('%H:%M')
    except ValueError:
        return None

# Function to get Time Range for each Time value
def get_time_range(time_str):
    time_str = convert_to_time(time_str)
    if time_str is None:
        return None
    hour = int(time_str[:2])
    time_range_start = f"{hour:02}:00"
    time_range_end = f"{hour:02}:59"
    return f"{time_range_start}-{time_range_end}"

# Add new column Time Range
df_grouped['Time Range'] = df_grouped['Time'].apply(get_time_range)
df_grouped

,InvoiceID,StoreID,Date,Time,Barcode,Total Include VAT,Time Range
0,VN0001010101230001,VN0001,2023-01-01,00:40:59,8935049500544,7000.0,00:00-00:59
1,VN0001010101230002,VN0001,2023-01-01,00:41:18,8938512632025,12000.0,00:00-00:59
2,VN0001010101230003,VN0001,2023-01-01,00:41:55,8936011773416,115000.0,00:00-00:59
3,VN0001010101230004,VN0001,2023-01-01,00:43:12,8801100128845,191000.0,00:00-00:59
4,VN0001010101230005,VN0001,2023-01-01,00:44:22,8934588843051,26000.0,00:00-00:59
...,...,...,...,...,...,...,...
15597313,VN9996023101230489,VN9996,2023-01-31,23:10:23,8936079121822,175000.0,23:00-23:59
15597314,VN9996023101230490,VN9996,2023-01-31,23:11:47,8935112200005,13000.0,23:00-23:59
15597315,VN9996023101230491,VN9996,2023-01-31,23:12:41,2010101000005,3000.0,23:00-23:59
15597316,VN9996023101230492,VN9996,2023-01-31,23:12:56,8936031835118,47000.0,23:00-23:59


In [15]:
# Create a pivot table to get the matrix representation
pivot_table_total = df_grouped.pivot_table(index=['StoreID','Date'], columns='Time Range', values='Total Include VAT', aggfunc='sum')
pivot_table_total = pivot_table_total.applymap("{:,.0f}".format)

pivot_table_total

Time Range         00:00-00:59 01:00-01:59 02:00-02:59 03:00-03:59   
StoreID Date                                                         
VN0001  2023-01-01   3,767,000   4,185,000     728,000     273,000  \
        2023-01-02     176,000     356,000     201,000     334,000   
        2023-01-03     496,000     446,000     129,000     106,000   
        2023-01-04     392,000     689,000      86,000         nan   
        2023-01-05     416,000     143,000     224,000     114,000   
...                        ...         ...         ...         ...   
VN9996  2023-01-28         nan         nan         nan         nan   
        2023-01-29         nan         nan         nan         nan   
        2023-01-30         nan         nan         nan         nan   
        2023-01-31         nan         nan         nan         nan   
        2023-02-01         nan         nan         nan         nan   

Time Range         04:00-04:59 05:00-05:59 06:00-06:59 07:00-07:59   
StoreID Date                                                         
VN0001  2023-01-01      66,000     126,000     371,000     451,000  \
        2023-01-02     218,000      22,000     319,000     393,000   
        2023-01-03      66,000     239,000     539,000   2,179,000   
        2023-01-04      76,000     623,000     467,000   2,745,000   
        2023-01-05     278,000       6,000   1,064,000   2,406,000   
...                        ...         ...         ...         ...   
VN9996  2023-01-28         nan         nan      82,000     394,000   
        2023-01-29         nan         nan         nan     162,000   
        2023-01-30         nan         nan      82,000     374,000   
        2023-01-31         nan         nan     144,000     117,000   
        2023-02-01         nan         nan         nan     161,000   

Time Range         08:00-08:59 09:00-09:59  ... 14:00-14:59 15:00-15:59   
StoreID Date                                ...                           
VN0001  2023-01-01     243,000     653,000  ...   1,561,000     632,000  \
        2023-01-02     569,000     550,000  ...   1,260,000     459,000   
        2023-01-03   3,356,000   1,604,000  ...   1,153,000   2,018,000   
        2023-01-04   3,318,000   2,476,000  ...     935,200   1,903,000   
        2023-01-05   4,623,000   2,333,000  ...   1,825,000   1,648,000   
...                        ...         ...  ...         ...         ...   
VN9996  2023-01-28   1,147,000     699,000  ...     836,000   2,682,000   
        2023-01-29   1,555,000   1,355,000  ...     695,000   1,648,000   
        2023-01-30     714,000     948,000  ...     503,000   1,039,000   
        2023-01-31     420,000     155,000  ...     319,000     958,000   
        2023-02-01     524,000     656,000  ...     468,000   1,381,000   

Time Range         16:00-16:59 17:00-17:59 18:00-18:59 19:00-19:59   
StoreID Date                                                         
VN0001  2023-01-01     751,000   1,950,000     794,000     763,000  \
        2023-01-02   1,175,000     472,000     343,000     367,000   
        2023-01-03   1,652,000   2,272,000   1,213,000   1,357,000   
        2023-01-04   1,661,000   2,384,000   1,199,300   1,545,000   
        2023-01-05   1,745,000   2,232,000   3,396,000   1,922,000   
...                        ...         ...         ...         ...   
VN9996  2023-01-28   3,178,000   3,658,000   3,685,000   3,803,000   
        2023-01-29   2,644,000   3,702,000   4,032,000   4,098,000   
        2023-01-30   2,838,000   3,150,000   2,305,000   3,108,000   
        2023-01-31   1,825,000   2,676,000   2,710,000   2,391,000   
        2023-02-01   1,569,000         nan         nan         nan   

Time Range         20:00-20:59 21:00-21:59 22:00-22:59 23:00-23:59  
StoreID Date                                                        
VN0001  2023-01-01   1,573,000   1,691,000   2,327,000   1,544,000  
        2023-01-02     848,000   1,173,000   1,012,000     660,000  
        2023-01

In [16]:
# Save the pivot table to csv file
# pivot_table_total.to_csv('TotalPerHour.csv')

In [17]:
# Create a pivot table to group the data by 'StoreID', 'WeekDay', and 'Time Range'
pivot_table_bill = df_grouped.pivot_table(index=['StoreID', 'Date'], columns='Time Range', values='InvoiceID', aggfunc=pd.Series.nunique, fill_value=0)


# Display the updated DataFrame
pivot_table_bill


Time Range          00:00-00:59  01:00-01:59  02:00-02:59  03:00-03:59   
StoreID Date                                                             
VN0001  2023-01-01           65           80           12            7  \
        2023-01-02            4            3            4            3   
        2023-01-03            8            5            5            1   
        2023-01-04            8            6            2            0   
        2023-01-05            4            4            2            4   
...                         ...          ...          ...          ...   
VN9996  2023-01-28            0            0            0            0   
        2023-01-29            0            0            0            0   
        2023-01-30            0            0            0            0   
        2023-01-31            0            0            0            0   
        2023-02-01            0            0            0            0   

Time Range          04:00-04:59  05:00-05:59  06:00-06:59  07:00-07:59   
StoreID Date                                                             
VN0001  2023-01-01            2            3            9            6  \
        2023-01-02            6            2           11           14   
        2023-01-03            2            3           18           78   
        2023-01-04            1           11           18           83   
        2023-01-05            2            1           26           86   
...                         ...          ...          ...          ...   
VN9996  2023-01-28            0            0            3           13   
        2023-01-29            0            0            0            4   
        2023-01-30            0            0            2           13   
        2023-01-31            0            0            2            5   
        2023-02-01            0            0            0            8   

Time Range          08:00-08:59  09:00-09:59  ...  14:00-14:59  15:00-15:59   
StoreID Date                                  ...                             
VN0001  2023-01-01            9           17  ...           21           15  \
        2023-01-02           15            9  ...           20           13   
        2023-01-03           95           54  ...           31           49   
        2023-01-04           99           73  ...           26           42   
        2023-01-05          112           66  ...           37           41   
...                         ...          ...  ...          ...          ...   
VN9996  2023-01-28           19           15  ...           29           56   
        2023-01-29           44           38  ...           21           44   
        2023-01-30           18           25  ...           11           29   
        2023-01-31           14            5  ...            9           34   
        2023-02-01           15           19  ...           17           26   

Time Range          16:00-16:59  17:00-17:59  18:00-18:59  19:00-19:59   
StoreID Date                                                             
VN0001  2023-01-01           17           19           24           14  \
        2023-01-02           26           11           14           14   
        2023-01-03           41           48           35           28   
        2023-01-04           44           53           28           21   
        2023-01-05           52           33           22           31   
...                         ...          ...          ...          ...   
VN9996  2023-01-28           74           81           85           97   
        2023-01-29           70           79          101           85   
        2023-01-30           64           83           63           70   
        2023-01-31           48           58           69           64   
        2023-02-01           32            0            0            0   

Time Range          20:00-20:59  21:00-21:59  22:00-22:59  23:00-23:59  
StoreID Da

In [ ]:
# Save the pivot table to csv file
# pivot_table_bill.to_csv('BillPerHour.csv')

In [18]:
# Replace non-numeric values (strings) with NaN in pivot_table_total and pivot_table_bill
pivot_table_total_numeric = pivot_table_total.applymap(lambda x: float(x.replace(',', '')) if isinstance(x, str) else np.nan)
pivot_table_bill_numeric = pivot_table_bill.applymap(float)

# Perform element-wise division between pivot_table_total_numeric and pivot_table_bill_numeric
result_df = pivot_table_total_numeric.div(pivot_table_bill_numeric)

# If you want to format the result as well, you can apply the formatting here
result_df = result_df.applymap("{:,.0f}".format)

# Display the resulting DataFrame
print(result_df)

Time Range         00:00-00:59 01:00-01:59 02:00-02:59 03:00-03:59   
StoreID Date                                                         
VN0001  2023-01-01      57,954      52,312      60,667      39,000  \
        2023-01-02      44,000     118,667      50,250     111,333   
        2023-01-03      62,000      89,200      25,800     106,000   
        2023-01-04      49,000     114,833      43,000         nan   
        2023-01-05     104,000      35,750     112,000      28,500   
...                        ...         ...         ...         ...   
VN9996  2023-01-28         nan         nan         nan         nan   
        2023-01-29         nan         nan         nan         nan   
        2023-01-30         nan         nan         nan         nan   
        2023-01-31         nan         nan         nan         nan   
        2023-02-01         nan         nan         nan         nan   

Time Range         04:00-04:59 05:00-05:59 06:00-06:59 07:00-07:59   
StoreID Date       

In [19]:
# Group data by 'Time Range', 'StoreID', and 'Barcode', and calculate the count of each barcode in each group
barcode_counts = df_grouped.groupby(['Time Range', 'StoreID', 'Barcode']).size().reset_index(name='Count')

# Sort the data by the count in descending order
sorted_barcode_counts = barcode_counts.sort_values(by='Count', ascending=False)

# Get the top 10 barcodes with the highest counts
top_10_barcodes = sorted_barcode_counts.head(10)

print(top_10_barcodes)


          Time Range StoreID        Barcode  Count
1742154  12:00-12:59  VN0134  2010101000005   2549
2120718  14:00-14:59  VN0134  2010101000005   2385
1093842  09:00-09:59  VN0060  8934588063053   2026
1742557  12:00-12:59  VN0134  8934588063053   1901
1675299  12:00-12:59  VN0060  8934588063053   1725
2121181  14:00-14:59  VN0134  8934588063053   1662
734314   07:00-07:59  VN0036  2010101000005   1460
1131234  09:00-09:59  VN0103  8934588063053   1453
2247033  15:00-15:59  VN0060  8934588063053   1425
971402   08:00-08:59  VN0134  8934588063053   1393


In [20]:
df_grouped

,InvoiceID,StoreID,Date,Time,Barcode,Total Include VAT,Time Range
0,VN0001010101230001,VN0001,2023-01-01,00:40:59,8935049500544,7000.0,00:00-00:59
1,VN0001010101230002,VN0001,2023-01-01,00:41:18,8938512632025,12000.0,00:00-00:59
2,VN0001010101230003,VN0001,2023-01-01,00:41:55,8936011773416,115000.0,00:00-00:59
3,VN0001010101230004,VN0001,2023-01-01,00:43:12,8801100128845,191000.0,00:00-00:59
4,VN0001010101230005,VN0001,2023-01-01,00:44:22,8934588843051,26000.0,00:00-00:59
...,...,...,...,...,...,...,...
15597313,VN9996023101230489,VN9996,2023-01-31,23:10:23,8936079121822,175000.0,23:00-23:59
15597314,VN9996023101230490,VN9996,2023-01-31,23:11:47,8935112200005,13000.0,23:00-23:59
15597315,VN9996023101230491,VN9996,2023-01-31,23:12:41,2010101000005,3000.0,23:00-23:59
15597316,VN9996023101230492,VN9996,2023-01-31,23:12:56,8936031835118,47000.0,23:00-23:59


In [22]:
# Backup df_grouped to df_grouped_bk
df_grouped_bk = df_grouped

df_grouped_bk

,InvoiceID,StoreID,Date,Time,Barcode,Total Include VAT,Time Range
0,VN0001010101230001,VN0001,2023-01-01,00:40:59,8935049500544,7000.0,00:00-00:59
1,VN0001010101230002,VN0001,2023-01-01,00:41:18,8938512632025,12000.0,00:00-00:59
2,VN0001010101230003,VN0001,2023-01-01,00:41:55,8936011773416,115000.0,00:00-00:59
3,VN0001010101230004,VN0001,2023-01-01,00:43:12,8801100128845,191000.0,00:00-00:59
4,VN0001010101230005,VN0001,2023-01-01,00:44:22,8934588843051,26000.0,00:00-00:59
...,...,...,...,...,...,...,...
15597313,VN9996023101230489,VN9996,2023-01-31,23:10:23,8936079121822,175000.0,23:00-23:59
15597314,VN9996023101230490,VN9996,2023-01-31,23:11:47,8935112200005,13000.0,23:00-23:59
15597315,VN9996023101230491,VN9996,2023-01-31,23:12:41,2010101000005,3000.0,23:00-23:59
15597316,VN9996023101230492,VN9996,2023-01-31,23:12:56,8936031835118,47000.0,23:00-23:59


In [23]:
from datetime import datetime
import pandas as pd

# Use backup DataFrame
df_grouped = df_grouped_bk
# Create the DataFrame
df = df_grouped

# Function to convert time string to a proper time format
def convert_to_time(time_str):
    try:
        return datetime.strptime(time_str, '%H:%M:%S').strftime('%H:%M')
    except ValueError:
        return None

# Function to get Time Range for each Time value
def get_time_range(time_str):
    time_str = convert_to_time(time_str)
    if time_str is None:
        return None
    hour = int(time_str[:2])
    time_range_start = f"{hour:02}:00"
    time_range_end = f"{hour:02}:59"
    return f"{time_range_start}-{time_range_end}"

# Add new column Time Range
df['Time Range'] = df['Time'].apply(get_time_range)

# Now, let's find the top 10 barcodes for each time range
# Group the data by 'StoreID', 'Date', 'Time Range', and 'Barcode', and count the occurrences
df_grouped = df.groupby(['StoreID', 'Date', 'Time Range', 'Barcode']).size().reset_index(name='Count')

# Sort the data by 'StoreID', 'Date', 'Time Range', and the count in descending order
df_grouped = df_grouped.sort_values(by=['StoreID', 'Date', 'Time Range', 'Count'], ascending=[True, True, True, False])

# Add the 'Rank' column based on the count
df_grouped['Rank'] = df_grouped.groupby(['StoreID', 'Date', 'Time Range'])['Count'].rank(ascending=False, method='first')

# Filter the top 10 barcodes for each time range
top_10_barcodes_all_time_ranges = df_grouped[df_grouped['Rank'] <= 10]

# Display the result
top_10_barcodes_all_time_ranges


,StoreID,Date,Time Range,Barcode,Count,Rank
47,VN0001,2023-01-01,00:00-00:59,8938512632025,4,1.0
0,VN0001,2023-01-01,00:00-00:59,2010101000005,3,2.0
20,VN0001,2023-01-01,00:00-00:59,8934588843051,3,3.0
31,VN0001,2023-01-01,00:00-00:59,8935049500544,3,4.0
34,VN0001,2023-01-01,00:00-00:59,8935049501503,3,5.0
...,...,...,...,...,...,...
13680552,VN9996,2023-02-01,16:00-16:59,2010403000185,1,6.0
13680553,VN9996,2023-02-01,16:00-16:59,2010805000028,1,7.0
13680554,VN9996,2023-02-01,16:00-16:59,8850157400107,1,8.0
13680555,VN9996,2023-02-01,16:00-16:59,8858223013039,1,9.0


In [24]:
df_grouped

,StoreID,Date,Time Range,Barcode,Count,Rank
47,VN0001,2023-01-01,00:00-00:59,8938512632025,4,1.0
0,VN0001,2023-01-01,00:00-00:59,2010101000005,3,2.0
20,VN0001,2023-01-01,00:00-00:59,8934588843051,3,3.0
31,VN0001,2023-01-01,00:00-00:59,8935049500544,3,4.0
34,VN0001,2023-01-01,00:00-00:59,8935049501503,3,5.0
...,...,...,...,...,...,...
13680564,VN9996,2023-02-01,16:00-16:59,8935049501503,1,16.0
13680567,VN9996,2023-02-01,16:00-16:59,8936079140014,1,17.0
13680568,VN9996,2023-02-01,16:00-16:59,8936079140076,1,18.0
13680569,VN9996,2023-02-01,16:00-16:59,8936127791014,1,19.0


In [25]:
test = df_grouped[(df_grouped['StoreID'] == 'VN0043') & (df_grouped['Date'] == '2023-01-06') & (df_grouped['Time Range'] == '18:00-18:59')]
test

,StoreID,Date,Time Range,Barcode,Count,Rank
2379475,VN0043,2023-01-06,18:00-18:59,8934588063053,3,1.0
2379471,VN0043,2023-01-06,18:00-18:59,2010902000228,2,2.0
2379489,VN0043,2023-01-06,18:00-18:59,8936185350130,2,3.0
2379465,VN0043,2023-01-06,18:00-18:59,89686043686,1,4.0
2379466,VN0043,2023-01-06,18:00-18:59,770795005619,1,5.0
2379467,VN0043,2023-01-06,18:00-18:59,2010805000202,1,6.0
2379468,VN0043,2023-01-06,18:00-18:59,2010805000226,1,7.0
2379469,VN0043,2023-01-06,18:00-18:59,2010805000356,1,8.0
2379470,VN0043,2023-01-06,18:00-18:59,2010805000363,1,9.0
2379472,VN0043,2023-01-06,18:00-18:59,2260101000134,1,10.0


In [34]:
test = test.merge(df_cate, on='Barcode', how='left')
test.head()

,StoreID,Date,Time Range,Barcode,Count,Rank,Division Name_x,Category Name_x,SubCategory Name_x,Division Name_y,Category Name_y,SubCategory Name_y
0,VN0043,2023-01-06,18:00-18:59,8934588063053,3,1.0,Beverage,Bottled Water,Purified water,Beverage,Bottled Water,Purified water
1,VN0043,2023-01-06,18:00-18:59,2010902000228,2,2.0,FF Offsite,Rice Offsite,Onigiri,FF Offsite,Rice Offsite,Onigiri
2,VN0043,2023-01-06,18:00-18:59,8936185350130,2,3.0,Beverage,Others Beverage,Others Drinks,Beverage,Others Beverage,Others Drinks
3,VN0043,2023-01-06,18:00-18:59,89686043686,1,4.0,Grocery,Instant Noodles,Instant Noodles Pack,Grocery,Instant Noodles,Instant Noodles Pack
4,VN0043,2023-01-06,18:00-18:59,770795005619,1,5.0,Snacks,Snacks for Drinks,Nuts/Seeds,Snacks,Snacks for Drinks,Nuts/Seeds


In [27]:
# query to get top 10 barcodes for time range 23:00-23:59 and store VN0001
df_query = top_10_barcodes_all_time_ranges[(top_10_barcodes_all_time_ranges['Time Range'] == '18:00-18:59') & (top_10_barcodes_all_time_ranges['StoreID'] == 'VN0043') & (top_10_barcodes_all_time_ranges['Date'] == '2023-01-06')]


In [28]:
df_query

,StoreID,Date,Time Range,Barcode,Count,Rank
2379475,VN0043,2023-01-06,18:00-18:59,8934588063053,3,1.0
2379471,VN0043,2023-01-06,18:00-18:59,2010902000228,2,2.0
2379489,VN0043,2023-01-06,18:00-18:59,8936185350130,2,3.0
2379465,VN0043,2023-01-06,18:00-18:59,89686043686,1,4.0
2379466,VN0043,2023-01-06,18:00-18:59,770795005619,1,5.0
2379467,VN0043,2023-01-06,18:00-18:59,2010805000202,1,6.0
2379468,VN0043,2023-01-06,18:00-18:59,2010805000226,1,7.0
2379469,VN0043,2023-01-06,18:00-18:59,2010805000356,1,8.0
2379470,VN0043,2023-01-06,18:00-18:59,2010805000363,1,9.0
2379472,VN0043,2023-01-06,18:00-18:59,2260101000134,1,10.0


In [29]:
test_df = df_grouped_bk
test_df_query = test_df[(test_df['StoreID'] == 'VN0043') & (test_df['Date'] == '2023-01-06') & (test_df['Time Range'] == '18:00-18:59')]

test_df_query

,InvoiceID,StoreID,Date,Time,Barcode,Total Include VAT,Time Range
2780878,VN0043010601230285,VN0043,2023-01-06,18:02:40,8935049500711,89000.0,18:00-18:59
2780879,VN0043010601230286,VN0043,2023-01-06,18:08:42,8938511021127,4000.0,18:00-18:59
2780880,VN0043010601230287,VN0043,2023-01-06,18:09:12,2010902000228,28000.0,18:00-18:59
2780881,VN0043010601230288,VN0043,2023-01-06,18:14:37,8934588063053,16000.0,18:00-18:59
2780882,VN0043010601230289,VN0043,2023-01-06,18:29:07,8935049501374,32000.0,18:00-18:59
2780883,VN0043010601230290,VN0043,2023-01-06,18:44:14,8934588063053,99000.0,18:00-18:59
2780884,VN0043010601230291,VN0043,2023-01-06,18:53:01,8936094291005,47000.0,18:00-18:59
2780885,VN0043010601230292,VN0043,2023-01-06,18:56:01,8935001701699,17000.0,18:00-18:59
2780886,VN0043010601230293,VN0043,2023-01-06,18:56:57,8934680033107,108000.0,18:00-18:59
2780887,VN0043010601230294,VN0043,2023-01-06,18:58:23,2010805000226,34000.0,18:00-18:59


In [32]:
df_cate.head()

,Barcode,Division Name,Category Name,SubCategory Name
0,9555715 201187,FF Onsite,Hot pot,Hot pot Thai
1,9555715 200067,FF Onsite,Hot pot,Hot pot Thai
2,8935282 907940,FoodService,Raw Material & Packages,Raw material
3,8935282 900651,FoodService,Raw Material & Packages,Raw material
4,7804350 046687,Wine&Liquor,Wine,Table/Varietal Wine


In [35]:
top_10_barcodes_all_time_ranges = top_10_barcodes_all_time_ranges.merge(df_cate, on='Barcode', how='left')
# Display head()

top_10_barcodes_all_time_ranges

,StoreID,Date,Time Range,Barcode,Count,Rank,Division Name,Category Name,SubCategory Name
0,VN0001,2023-01-01,00:00-00:59,8938512632025,4,1.0,Beverage,Bottled Water,Purified water
1,VN0001,2023-01-01,00:00-00:59,2010101000005,3,2.0,FF Beverage,Cold Beverages,Cup Of Ice
2,VN0001,2023-01-01,00:00-00:59,8934588843051,3,3.0,Beverage,Sports Drinks,Sports Drinks Sub
3,VN0001,2023-01-01,00:00-00:59,8935049500544,3,4.0,Beverage,Bottled Water,Mineral water
4,VN0001,2023-01-01,00:00-00:59,8935049501503,3,5.0,Beverage,Carbonated Drinks - CSD,Sugar CSD
...,...,...,...,...,...,...,...,...,...
7451027,VN9996,2023-02-01,16:00-16:59,2010403000185,1,6.0,FF Beverage,FF Coffee,Local Coffee
7451028,VN9996,2023-02-01,16:00-16:59,2010805000028,1,7.0,FF Onsite,Other Cuisine,Korean cuisine Sub
7451029,VN9996,2023-02-01,16:00-16:59,8850157400107,1,8.0,Snacks,Snacks for Drinks,Meat snacks
7451030,VN9996,2023-02-01,16:00-16:59,8858223013039,1,9.0,Snacks,Non-potato Chips,Alternative Snacks
